In [74]:
import pandas as pd
import numpy as np
from gensim.models import KeyedVectors
import re
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
from tensorflow.keras.preprocessing.text import Tokenizer
import datetime
from nltk.corpus import stopwords
import gensim
import smart_open
from string import punctuation
stop_words = stopwords.words('english')
from sklearn.model_selection import train_test_split
from gensim.models import Word2Vec
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Lambda
import tensorflow.keras.backend as K
from tensorflow.keras.optimizers import Adadelta
from tensorflow.keras.callbacks import ModelCheckpoint,Callback
from tensorflow.keras.layers import Input, Bidirectional, LSTM, dot, Flatten, Dense, Reshape, add, Dropout, BatchNormalization

In [52]:
df = pd.read_csv('train.csv')

In [53]:
df.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [5]:
word2vec = gensim.models.KeyedVectors.load_word2vec_format('C:\\Users\\Suraj\\GoogleNews-vectors-negative300.bin.gz', binary=True)

In [54]:
#Cleaning


SPECIAL_TOKENS = {
    'quoted': 'quoted_item',
    'non-ascii': 'non_ascii_word',
    'undefined': 'something'
}


def clean(text, stem_words=True):

    
    if pd.isnull(text):
        return ''

    if type(text) != str or text=='':
        return ''
    ps = PorterStemmer()
    
    text = text.lower()
    text = re.sub("\'s", " ", text) 
    text = re.sub(" whats ", " what is ", text, flags=re.IGNORECASE)
    text = re.sub("\'ve", " have ", text)
    text = re.sub("can't", "can not", text)
    text = re.sub("n't", " not ", text)
    text = re.sub("i'm", "i am", text, flags=re.IGNORECASE)
    text = re.sub("\'re", " are ", text)
    text = re.sub("\'d", " would ", text)
    text = re.sub("\'ll", " will ", text)
    text = re.sub("e\.g\.", " eg ", text, flags=re.IGNORECASE)
    text = re.sub("b\.g\.", " bg ", text, flags=re.IGNORECASE)
    text = re.sub("(\d+)(kK)", " \g<1>000 ", text)
    text = re.sub("e-mail", " email ", text, flags=re.IGNORECASE)
    text = re.sub("(the[\s]+|The[\s]+)?U\.S\.A\.", " America ", text, flags=re.IGNORECASE)
    text = re.sub("(the[\s]+|The[\s]+)?United State(s)?", " America ", text, flags=re.IGNORECASE)
    text = re.sub("\(s\)", " ", text, flags=re.IGNORECASE)
    text = re.sub("[c-fC-F]\:\/", " disk ", text)
    text = re.sub('(?<=[0-9])\,(?=[0-9])', "", text)
    text = re.sub('\$', " dollar ", text)
    text = re.sub('\%', " percent ", text)
    text = re.sub('\&', " and ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r'\d+', '',text)
    text = re.sub('(\\b[A-Za-z] \\b|\\b [A-Za-z]\\b)', '', text)
    text = text.replace("?","")
    text = text.replace("(","")
    text = text.replace(")","")
    text = text.replace('"',"")
    text = text.replace(",","")
    text = text.replace("#","")   
    text = text.replace("-","")    
    text = text.replace("..","")
    text = text.replace("/","")
    text = text.replace("\\","")
    text = text.replace(":","")
    text = text.replace("the","") 
        
    text = text.split()
    
    return text


In [55]:
df['question1'] = df['question1'].apply(clean)
df['question2'] = df['question2'].apply(clean)

In [56]:
df.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,"[what, is, step, by, step, guide, to, invest, ...","[what, is, step, by, step, guide, to, invest, ...",0
1,1,3,4,"[what, is, story, of, kohinoor, kohinoor, diam...","[what, would, happen, if, indian, government, ...",0
2,2,5,6,"[how, can, increase, speed, of, my, internet, ...","[how, can, internet, speed, be, increased, by,...",0
3,3,7,8,"[why, am, mentally, very, lonely, how, can, so...","[find, remainder, when, [math]^{}[math], is, d...",0
4,4,9,10,"[which, one, dissolve, in, water, quikly, suga...","[which, fish, would, survive, in, salt, water]",0


In [57]:
vocabulary = dict()
inverse_vocabulary = ['<unk>']
questions_cols = ['question1', 'question2']

In [58]:
for index,row in df.iterrows():
    # Iterate through the text of both questions of the row
    for question in questions_cols:
        q2n = []  # q2n -> question numbers representation
        for word in row[question]:
            # Check for unwanted words
            if word in stop_words and word not in word2vec.vocab:
                continue
            if word not in vocabulary:
                vocabulary[word] = len(inverse_vocabulary)
                q2n.append(len(inverse_vocabulary))
                inverse_vocabulary.append(word)
            else:
                q2n.append(vocabulary[word])
        df.at[index, question] = q2n 

In [59]:
vocabulary

{'what': 1,
 'is': 2,
 'step': 3,
 'by': 4,
 'guide': 5,
 'invest': 6,
 'in': 7,
 'share': 8,
 'market': 9,
 'india': 10,
 'story': 11,
 'kohinoor': 12,
 'diamond': 13,
 'would': 14,
 'happen': 15,
 'if': 16,
 'indian': 17,
 'government': 18,
 'stole': 19,
 'back': 20,
 'how': 21,
 'can': 22,
 'increase': 23,
 'speed': 24,
 'my': 25,
 'internet': 26,
 'connection': 27,
 'while': 28,
 'using': 29,
 'vpn': 30,
 'be': 31,
 'increased': 32,
 'hacking': 33,
 'through': 34,
 'dns': 35,
 'why': 36,
 'am': 37,
 'mentally': 38,
 'very': 39,
 'lonely': 40,
 'solve': 41,
 'it': 42,
 'find': 43,
 'remainder': 44,
 'when': 45,
 '[math]^{}[math]': 46,
 'divided': 47,
 'which': 48,
 'one': 49,
 'dissolve': 50,
 'water': 51,
 'quikly': 52,
 'sugar': 53,
 'salt': 54,
 'methane': 55,
 'carbon': 56,
 'di': 57,
 'oxide': 58,
 'fish': 59,
 'survive': 60,
 'astrology': 61,
 'capricorn': 62,
 'sun': 63,
 'cap': 64,
 'moon': 65,
 'rising.what': 66,
 'does': 67,
 'that': 68,
 'say': 69,
 'about': 70,
 'me': 71

In [60]:
len(vocabulary)

116885

In [61]:
df.drop(['id','qid1','qid2'],axis = 1,inplace = True)

In [62]:
df.head()

,question1,question2,is_duplicate
0,"[1, 2, 3, 4, 3, 5, 6, 7, 8, 9, 7, 10]","[1, 2, 3, 4, 3, 5, 6, 7, 8, 9]",0
1,"[1, 2, 11, 12, 12, 13]","[1, 14, 15, 16, 17, 18, 19, 12, 12, 13, 20]",0
2,"[21, 22, 23, 24, 25, 26, 27, 28, 29, 30]","[21, 22, 26, 24, 31, 32, 4, 33, 34, 35]",0
3,"[36, 37, 38, 39, 40, 21, 22, 41, 42]","[43, 44, 45, 46, 2, 47, 4]",0
4,"[48, 49, 50, 7, 51, 52, 53, 54, 55, 56, 57, 58]","[48, 59, 14, 60, 7, 54, 51]",0


In [63]:
embedding_dim = 300
embeddings = 1 * np.random.randn(len(vocabulary) + 1, embedding_dim)
embeddings[0] = 0
# Build the embedding matrix
for word, index in vocabulary.items():
    if word in word2vec.vocab:
        embeddings[index] = word2vec.word_vec(word)

del word2vec

In [64]:
len(embeddings)

116886

In [65]:
embeddings[1]

array([ 0.13964844, -0.00616455,  0.21484375,  0.07275391, -0.16113281,
        0.07568359,  0.16796875, -0.20117188,  0.12597656,  0.00915527,
        0.05249023, -0.15136719, -0.02758789,  0.04199219, -0.234375  ,
        0.13867188, -0.02600098,  0.07910156,  0.02746582, -0.13085938,
       -0.02478027,  0.10009766, -0.07910156, -0.07714844,  0.03759766,
        0.16894531,  0.05371094, -0.05200195,  0.14453125, -0.04370117,
       -0.12597656,  0.06884766, -0.10595703, -0.14550781, -0.00331116,
        0.01367188,  0.13964844,  0.01660156,  0.03417969,  0.16113281,
       -0.01080322,  0.06689453,  0.06835938, -0.15136719, -0.16894531,
        0.03295898, -0.06884766,  0.06787109, -0.07373047,  0.08300781,
        0.05761719,  0.14550781, -0.11865234, -0.13671875,  0.12402344,
        0.04296875, -0.11962891, -0.08154297,  0.06494141, -0.05639648,
       -0.04394531,  0.1484375 , -0.07714844,  0.04614258, -0.02624512,
       -0.06591797,  0.04980469,  0.08886719, -0.01647949, -0.02

In [66]:
question1 = []
question2 = []
is_duplicate = []
for index,row in df.iterrows():
    print(row['question1'])
    print(row['question2'])
    print(row['is_duplicate'])
    break

[1, 2, 3, 4, 3, 5, 6, 7, 8, 9, 7, 10]
[1, 2, 3, 4, 3, 5, 6, 7, 8, 9]
0


In [67]:
maxlen = max(df['question1'].map(lambda x: len(x)).max(),df['question2'].map(lambda x:len(x)).max())
print(maxlen)

191


In [70]:
MAX_SEQUENCE_LENGTH = 100
q1_data = pad_sequences(df['question1'], maxlen=MAX_SEQUENCE_LENGTH)
q2_data = pad_sequences(df['question2'], maxlen=MAX_SEQUENCE_LENGTH)
labels = np.array(df['is_duplicate'], dtype=int)
print('Shape of question1 data tensor:', q1_data.shape)
print('Shape of question2 data tensor:', q2_data.shape)
print('Shape of label tensor:', labels.shape)

Shape of question1 data tensor: (404290, 100)
Shape of question2 data tensor: (404290, 100)
Shape of label tensor: (404290,)


In [71]:
q1_data[0]

array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  1,  2,  3,  4,  3,  5,  6,  7,  8,  9,  7, 10])

In [73]:
Q1_TRAINING_DATA_FILE = 'q1_train.npy'
Q2_TRAINING_DATA_FILE = 'q2_train.npy'
LABEL_TRAINING_DATA_FILE = 'label_train.npy'
WORD_EMBEDDING_MATRIX_FILE = 'word_embedding_matrix.npy'

np.save(open(Q1_TRAINING_DATA_FILE, 'wb'), q1_data)
np.save(open(Q2_TRAINING_DATA_FILE, 'wb'), q2_data)
np.save(open(LABEL_TRAINING_DATA_FILE, 'wb'), labels)
np.save(open(WORD_EMBEDDING_MATRIX_FILE, 'wb'), embeddings)

In [75]:
word_embedding_matrix = np.load(open(WORD_EMBEDDING_MATRIX_FILE, 'rb'))